In [50]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [51]:
from fuzzywuzzy import process

In [52]:
url = 'https://qz.com/1735700/apple-and-disney-are-creating-an-explosion-of-tv-series-budgets/'

In [53]:
res = requests.get(url)
soup = BeautifulSoup(res.text,'html.parser')

In [54]:
table = soup.find(name='table',attrs={'class':'c188c'})

In [55]:
df_table = pd.read_html(str(table))[0]

In [56]:
choices = ['Action','Adventure', 'Drama', 'Comedy', 'Thriller/Suspense',
       'Horror', 'Romantic Comedy', 'Musical', 'Documentary',
       'Black Comedy', 'Western', 'Concert/Performance',
       'Multiple Genres', 'Reality']

In [57]:
#https://www.rottentomatoes.com/tv/the_morning_show/
drama_url = 'https://www.rottentomatoes.com/tv/'
j=0
for i in df_table['TV series']:
    formated_i = i.lower().replace(' ','_')
    full_url = drama_url+formated_i
    #print(full_url)
    res = requests.get(full_url)
    
    if res.ok:
        series_soup = BeautifulSoup(res.text,'html.parser')
        rows = series_soup.find_all(name='td',attrs={'class':'fgm'})
        table_right=rows[0].findParent().findParent() #td->tr->tbody
        table_right_data = table_right.find_all(name='td')
        genre = table_right_data[5].text
        
    else:
        genre = 'Other'
        
    final_genre = process.extractOne(genre, choices)[0]
    df_table.at[j,'Genre']=final_genre       
    j+=1

In [59]:
df_table['Estimated cost per episode'] = df_table['Estimated cost per episode'].replace({'\$': ''}, regex=True)

In [61]:
df_table['Estimated cost per episode'] = df_table['Estimated cost per episode'].replace({'million': ''}, regex=True)

In [64]:
 df_table.rename(columns={"Estimated cost per episode": "Estimatedcostperepisode(million)"},inplace=True)

In [65]:
df_table

,TV series,Network,Years on air,Estimatedcostperepisode(million),Genre
0,Upcoming Marvel shows,Disney+,2020-,25,Thriller/Suspense
1,See,Apple TV+,2019-,15-17,Drama
2,The Morning Show,Apple TV+,2019-,15,Drama
3,The Mandalorian,Disney+,2019-,15,Action
4,Game of Thrones,HBO,2011-2019,15,Action
5,The Crown,Netflix,2016-,13,Drama
6,Westworld,HBO,2016-,10,Action
7,Friends,NBC,1994-2004,10,Comedy
8,Jack Ryan,Amazon,2018-,8,Action
9,American Gods,Starz,2017-,8,Action


In [19]:
df=df_table.groupby(['Genre']).count()
df

,TV series,Network,Years on air,Estimated cost per episode
Genre,,,,
Action,8,8,8,8
Comedy,2,2,2,2
Drama,11,11,11,11
Horror,1,1,1,1
Thriller/Suspense,1,1,1,1
Western,1,1,1,1


In [13]:
import mysql.connector
from mysql.connector import errorcode
import config
cnx = mysql.connector.connect(
    host = config.my_cred['host'],
    user = config.my_cred['user'],
    passwd = config.my_cred['passwd']
)
print(config.my_cred['user'])
print(cnx)

admin


In [14]:
cursor = cnx.cursor()

In [15]:
db_name = 'serials'

In [16]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [17]:
try:
    cursor.execute("USE {}".format(db_name))

#if the previous line fails because there isn't a db by that name run this line

except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database serials does not exists.
Database serials created successfully.


In [21]:
DB_NAME = 'serials'

TABLES = {}
TABLES['serials_genre'] = (
    "CREATE_TABLE serials_genre ("
    " TV_series varchar(20) NOT NULL,"
    " Network varchar(190) NOT NULL,"
    " Years_on_air varchar(190) NOT NULL,"
    " Estimated_cost_per_episode varchar(50) NOT NULL,"
    " Genre varchar(50) NOT NULL"
    ") ENGINE=InnoDB")

In [22]:
try:
    cursor.execute("USE {}".format(DB_NAME))

#if the previous line fails because there isn't a db by that name run this line

except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, DB_NAME)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)
        
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

# cursor.close()
# cnx.close()

Creating table serials_genre: You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'CREATE_TABLE serials_genre ( TV_series varchar(20) NOT NULL, Network varchar(190' at line 1
